In [1]:
import json
from web3 import Web3
import os
import platform
from getpass import getpass
import random

In [80]:
operating_system = platform.system()
d = '/'
if operating_system == 'Windows':
    d = '\\'

ganache_url = "HTTP://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# This is our main account that controls everything
web3.eth.defaultAccount = web3.eth.accounts[0]

In [81]:
web3.eth.accounts

['0xb6ef75b96D856a6cE0C796be4a7ABfB81E5D600A',
 '0xa2D0B6dbCd04a789da44CCECA9d66B6dfc93A124',
 '0xEE4F5078d802207d68eE9668AE6b82FCFCF7e290',
 '0x9cbFF84198C1c3ed898C52Fbc0076Bf008476C4B',
 '0x18a31E094820e4C9C79a073A0927FA72635b18C7',
 '0xDED00A7f19D445B2c6D3ddFffbf78BB490771937',
 '0x37594b8C8aC1D94457C86e30dE793099CE659817',
 '0x6a1ab8E6bb7e9CC3463997FC7F38e3B3855282b5',
 '0xAfd3bB14ff7735B0D9792e5bcc9BB5e758a32ccB',
 '0x8b23fa43919cB11b9CDa04FA45B92aaA0A0Adcd8']

In [82]:
def getContract(filename):
    with open('..'+ d +'build'+ d +'contracts'+ d +filename) as contractFile:
        contractJson=json.load(contractFile)
        contractABI =contractJson['abi']
        contractAdd=web3.toChecksumAddress(contractJson['networks']['5777']['address']) 
        return contractABI, contractAdd



# Get the organization contract address
organizationContractABI, organizationContractAdd = getContract('Organization.json')	
organizationContract = web3.eth.contract(address=organizationContractAdd, abi=organizationContractABI)

#get the user contract address
userContractABI,userContractAdd = getContract('User.json')
userContract = web3.eth.contract(address=userContractAdd, abi=userContractABI)

# Get the accounts contract address
accountsContractABI,accountsContractAdd = getContract('Accounts.json')
accountsConract = web3.eth.contract(address=accountsContractAdd, abi=accountsContractABI)

loansContractABI, loansContractAddress= getContract('Loans.json')


In [83]:
# This should be changed to be a more robus way. We can give the loansContractAddress using the constructor
# but this has problems when doing this in the 2_deploy_contracts.js
userContract.functions.setLoansContractAddress(loansContractAddress).transact()
organizationContract.functions.setLoansContractAddress(loansContractAddress).transact()

HexBytes('0xb04079122d0a04a980339aba090c12d0c749e37d013bbdd87c7980cc6992c708')

HexBytes('0xce71fdec2c759ae103180c0556efcb9ec2f3eb6eb7cb917790727eaa9afc4580')

In [84]:
accountsConract.functions.add(web3.eth.accounts[1], False).transact()
accountsConract.functions.add(web3.eth.accounts[2], False).transact()
accountsConract.functions.add(web3.eth.accounts[5], True).transact()
accountsConract.functions.add(web3.eth.accounts[6], True).transact()

HexBytes('0x143514a9d2e685ff526b00b060e44676cebb18473f9b121f36de4ff2f2715730')

HexBytes('0x8930f4dcf350a19f88bd4b2464f72f2d752ae6ff42af2d74105e495360b89945')

HexBytes('0x85c9c56d5952447cc4a46f163d35dc9de59ff92873f80c047ea1007b54635a45')

HexBytes('0x9a22b95614e7240815339792d1b2cf1d6a609d51e16d79e4c4112c08e7fc5319')

In [85]:
def deleteUser():
    accountsConract.functions.deleteAccount(web3.eth.accounts[1]).transact()

def createLoan(_address,_loaner,_amount,_loansContractAddress):
    check=organizationContract.functions.createLoan(_address,_loaner,_amount).transact()
    return check

In [7]:
deleteUser()

In [86]:
def getPendingLoans(_userContract, _loanieAddress, _privateKey):
    transaction = _userContract.functions.getPendingLoans(
    ).buildTransaction({
    'gas': 70000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanieAddress,
    'nonce': web3.eth.getTransactionCount(_loanieAddress)
    }) 
    
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _userContract.events.getAmounts().processReceipt(receipt)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values


In [93]:
# Suppose bank wants to create a loan
loanieAddress = input("Enter the addrses: ")

### Check to see whether it is a user or organization address
index = accountsConract.functions.getIndex(loanieAddress).call()

Enter the addrses: 0x37594b8C8aC1D94457C86e30dE793099CE659817


In [68]:
index

1

## Get pending loans

In [79]:
# Get pending loans
def getPendingLoansList(_accountIndex, _userContract, _accountsContract, _loanieAddress):
    pendingLoans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if not loanieType:
            privateKey = getpass("Enter your password: ")
            values = getPendingLoans(_userContract, _loanieAddress, privateKey)
            for i in range(len(values['_amounts'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                pendingLoans.append(string)
    else:
        print("This account is not registered in our system.")
    return pendingLoans

In [77]:
getPendingLoansList(index, userContract, accountsConract, loanieAddress)

Enter your key: ········


['68889 0x37594b8C8aC1D94457C86e30dE793099CE659817 1588932225 ',
 '78657 0xDED00A7f19D445B2c6D3ddFffbf78BB490771937 1588932236 ',
 '28936 0x37594b8C8aC1D94457C86e30dE793099CE659817 1588932246 ',
 '0 0x0000000000000000000000000000000000000000 0 ']

## Confirm or reject loans

In [76]:
# Confirm or reject loans
pendingLoans = getPendingLoansList(index, userContract, accountsConract, loanieAddress)
private_key = getpass('Enter your key: ')
for pendingLoan in pendingLoans:
    print('for address: ' + pendingLoan.split(' ')[1])
    loanId = pendingLoan.split(' ')[2]
    choice = input('c/r?')
    if choice == 'c':
        transaction = userContract.functions.validateLoan(True,
        int(loanId)).buildTransaction({
        'gas': 70000,
        'gasPrice': web3.toWei('1', 'gwei'),
        'from': loanieAddress,
        'nonce': web3.eth.getTransactionCount(loanieAddress)
        }) 
        signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
        transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    elif choice == 'r':
        transaction = userContract.functions.validateLoan(False,
        int(loanId)).buildTransaction({
        'gas': 70000,
        'gasPrice': web3.toWei('1', 'gwei'),
        'from': loanieAddress,
        'nonce': web3.eth.getTransactionCount(loanieAddress)
        }) 
        signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
        transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    else:
        print("Not recognized operation.")

Enter your key: ········
Enter your key: ········
for address: 0xDED00A7f19D445B2c6D3ddFffbf78BB490771937
c/r?r
for address: 0x37594b8C8aC1D94457C86e30dE793099CE659817
c/r?l
Not recognized operation.
for address: 0xDED00A7f19D445B2c6D3ddFffbf78BB490771937
c/r?l
Not recognized operation.
for address: 0x37594b8C8aC1D94457C86e30dE793099CE659817
c/r?l
Not recognized operation.


# Create a new loan

In [94]:
# Create a new loan
if index != -1:
    loanieType = accountsConract.functions.getType(index).call()
    if  loanieType:
        if createLoan(web3.eth.accounts[2],loanieAddress,random.randint(1,100000),loansContractAddress):
            print("Loan created")
else:
    print("This account is not registered in our system.")

Loan created
